In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
1. Import the libraries, load dataset, print shape of data, data description. (5 Marks)
2. Understand of data-columns: (5 Marks)
a. Drop all other columns except “text” and “airline_sentiment”.
b. Check the shape of data.
c. Print first 5 rows of data.

In [ ]:
SEED = 42

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
FILE_NAME='Tweets.csv'

In [ ]:
data = pd.read_csv(FILE_NAME)

In [ ]:
data.shape

(14640, 15)

In [ ]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials to the experience... tacky.,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I need to take another trip!,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing about it,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
data['airline_sentiment'].value_counts(normalize=True).round(2)

negative    0.63
neutral     0.21
positive    0.16
Name: airline_sentiment, dtype: float64

In [ ]:
data['airline'].value_counts(normalize=True).round(4)

United            0.2611
US Airways        0.1990
American          0.1885
Southwest         0.1653
Delta             0.1518
Virgin America    0.0344
Name: airline, dtype: float64

In [ ]:
data['retweet_count'].value_counts(normalize=True).round(4)

0     0.9476
1     0.0437
2     0.0045
3     0.0015
4     0.0012
5     0.0003
7     0.0002
6     0.0002
22    0.0001
8     0.0001
32    0.0001
9     0.0001
31    0.0001
18    0.0001
15    0.0001
28    0.0001
44    0.0001
11    0.0001
Name: retweet_count, dtype: float64

In [ ]:
3. Text pre-processing: Data preparation. (20 Marks)
a. Html tag removal.
b. Tokenization.
c. Remove the numbers.
d. Removal of Special Characters and Punctuations.
e. Conversion to lowercase.
f. Lemmatize or stemming.
g. Join the words in the list to convert back to text string in the dataframe. (So that each row
contains the data in text format.)
h. Print first 5 rows of data after pre-processing.
4. Vectorization: (10 Marks)
a. Use CountVectorizer.
b. Use TfidfVectorizer.
5. Fit and evaluate model using both type of vectorization. (6+6 Marks)
6. Summarize your understanding of the application of Various Pre-processing and Vectorization and
performance of your model on this dataset. (8 Marks)

Benchmark

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [ ]:
message_text= data.loc[0, 'text']

In [ ]:
sid.polarity_scores(message_text)

{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}

In [ ]:
scores = data['text'].apply(lambda x: sid.polarity_scores(x)).apply(pd.Series)

In [ ]:
scores=scores.apply(pd.Series)

In [ ]:
scores.head()

In [ ]:
scores['sentiment']=data['airline_sentiment']

In [ ]:
scores.describe()

In [ ]:
def find_threshold(score,
                   actual_distribution={'negative':0.63,
                                        'nuetral':0.21,
                                        'positive':0.16},
                   ):
    total= len(score)

    for threshold in [x/100 for x in range(-100,100,1]):
      total_negatives = sum(score < threshold)
      if abs(total_negatives/total - actual_distribution['negative']) <= 0.01:
        negative_threshold = threshold
        break

    for threshold in [x/100 for x in range(int(negative_threshold*100,100,1)]:
      total_neutral = len(np.where((score >= negative_threshold) & score <=threshold))[0])
      if abs(total_neutral/total - actual_distribution['neutral']) <= 0.01:
        nuetral_threshold = threshold
        break

    return nuetral_threshold, negative_threshold                                 

SyntaxError: ignored

In [ ]:
find_threshold(scores['compound'].values)

NameError: ignored

In [ ]:
import time
start = time.time()
find_threshold(scores['compound'].values)
end = time.time()
print("{:.2f} seconds"}format(end-start))

In [ ]:
scores['p_sentiment'] = pd.cut(scores['compound']),
                                bins=[-1,0.2023,0.5766,1]
                                labels=['negative','nuetral','positive']

In [ ]:
scores.head()

In [ ]:
scores['flag'] = scores['p_sentiment']==scores['sentiment']

In [ ]:
scores['flag'].mean()

In [ ]:
scores.groupby(by='sentiment')['flag'].agg(['mean','count']).round(2)

Processing

In [1]:
# Import necessary libraries.
import re, string, unicodedata
import pandas as pd
import nltk           
                        # Natural language processing tool-kit
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

!pip install contractions
import contractions


from bs4 import BeautifulSoup                 # Beautiful soup is a parsing library that can use different parsers.
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords, wordnet    # Stopwords, and wordnet corpus
from nltk.stem import LancasterStemmer, WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
     |████████████████████████████████| 245kB 4.5MB/s 
     |████████████████████████████████| 317kB 25.5MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81699 sha256=6b4273fe5c44f7e0eab78064bd70234c401594e4aed544a45c22d293b668b1d7
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

# Perform the above operation over all the rows of text column of the dataframe.
for i, row in data.iterrows():
    text = data.at[i, 'text']
    clean_text = replace_contractions(text)
    data.at[i,'text'] = clean_text
data.head()

In [ ]:
# Tokenize the words of whole dataframe.
for i, row in data.iterrows():
    text = data.at[i, 'text']
    words = nltk.word_tokenize(text)
    data.at[i,'text'] = words
data.head()

In [ ]:
# save the stopwords in a list named stopwords.
stopwords = stopwords.words('english')
stopwords

In [ ]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []                        # Create empty list to store pre-processed words.
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)        # Append processed words to new list.
    return new_words

In [ ]:
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []                        # Create empty list to store pre-processed words.
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)    # Append processed words to new list.
    return new_words

In [ ]:
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []                        # Create empty list to store pre-processed words.
    for word in words:
        if word not in stopwords:
            new_words.append(word)        # Append processed words to new list.
    return new_words

In [ ]:
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []                            # Create empty list to store pre-processed words.
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)                # Append processed words to new list.
    return stems

In [ ]:
def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []                           # Create empty list to store pre-processed words.
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)              # Append processed words to new list.
    return lemmas

In [ ]:
def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

In [ ]:
# Iterate the normalize funtion over whole data.
for i, row in data.iterrows():
    words = data.at[i, 'text']
    words = normalize(words)
    data.at[i,'text'] = words
data.head()

In [ ]:
def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems, lemmas

In [ ]:
data['lemma'] = ''
data['stem'] = ''

for i, row in data.iterrows():
    words = data.at[i, 'text']
    stems, lemmas = stem_and_lemmatize(words)
    data.at[i,'stem'] = stems
    data.at[i, 'lemma'] = lemmas
data.head()

Processing

In [ ]:
ps = PorterStemmer()
def preprocess(text):
  '''Pre-processing steps as described above.'''
  # text = text.lower()
  # # text = re.sub(r'[.|,|)|(|\|/]',r'', text)        #Removing Punctuations
  # words = [word for word in text if word not in stopwords]   # removing stopwords
  # return words
  text = text.lower()                 # Converting to lowercase
  text = re.sub(r'[.|,|)|(|\|/]',r' ', text) # Removing punctuation
  word_tokens = word_tokenize(text)  # Tokenization

  words = [word for word in word_tokens if word not in stopwords] # Stop word removal
  return words

In [ ]:
Build count and tfidf

In [ ]:
X = data['text']
y = data.airline_sentiment

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)

In [ ]:
import xgboost as xgb

In [ ]:
cv = CountVectorizer()  
X = cv.fit_transform(sent)
print(cv.vocabulary_)
print(X.shape)
print(type(X))
print(X.toarray())

In [ ]:
myvocabulary = list(set(sent))
myvocabulary

In [ ]:
corpus = {1: d1, 2: d2, 3: d3, 4: d4}
tfidf = TfidfVectorizer(vocabulary = myvocabulary, ngram_range = (1,2))
tfs = tfidf.fit_transform(corpus.values())
feature_names = tfidf.get_feature_names()
corpus_index = [n for n in corpus]
import pandas as pd
df = pd.DataFrame(tfs.T.todense(), index=feature_names, columns=corpus_index)
print(tfidf.vocabulary_)
print(tfidf.idf_)
print(tfs.shape)
print(df)


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
sentiment = SentimentIntensityAnalyzer()
sentiment_dict1 = sentiment.polarity_scores(d1)
sentiment_dict2 = sentiment.polarity_scores(d2)
sentiment_dict3 = sentiment.polarity_scores(d3)
sentiment_dict4 = sentiment.polarity_scores(d4)

print(sentiment_dict1)
print(sentiment_dict2)
print(sentiment_dict3)
print(sentiment_dict4)

In [ ]:
from textblob import TextBlob
print(TextBlob(d1).sentiment)
print(TextBlob(d2).sentiment)
print(TextBlob(d3).sentiment)
print(TextBlob(d4).sentiment)